# Tarea Primer Red Neuronal 

Diego Lozoya Morales | 745345

# Paqueterías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from scipy import stats
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

colors = ["cornflowerblue", "indianred", "darkseagreen", "plum", "dimgray"]

# Funciones

In [ ]:
def calibracion(predicciones, y_test):
    df_calibracion = pd.DataFrame({
        'predicciones':predicciones,
        'y':y_test
    })
    df_calibracion['bins'] = pd.qcut(df_calibracion['predicciones'], q=15, labels=False)+1
    grouped = df_calibracion.groupby('bins').mean()

    plt.figure(figsize=(10, 5))
    plt.plot(grouped.predicciones, grouped.y, label="Modelo", marker="o", color=colors[0])
    plt.plot(grouped.predicciones, grouped.predicciones, label="Real", marker="o", color=colors[1])
    plt.xlabel("Probabilidad")
    plt.ylabel("Real")
    plt.title("Calibración")
    plt.grid()
    plt.legend()
    plt.show()

# Data

In [ ]:
data = pd.read_csv("Data/Boston.csv").drop('Unnamed: 0', axis=1, inplace=True)
data = pd.get_dummies(data, columns=['rad'])

## Variable dependiente

In [ ]:
target = 'medv'
y = data[target]

# Cross validation

In [ ]:
X = data.copy().drop(target, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Red Neuronal

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('nn', MLPRegressor())
])

params_grid = {
     'nn__hidden_layer_sizes': [(5,5,5), (10,), (100,), (10,10)],
     'nn__activation': ['tanh', 'relu', 'logistic'],
     'nn__alpha': [0.1, 0.01]
}

grid_nn = GridSearchCV(estimator=pipeline, n_jobs=-1, cv=5, scoring= 'r2', param_grid=params_grid)
grid_nn.fit(X_train, y_train)

In [ ]:
predicciones_nn = grid_nn.best_estimator_.predict(X_test)
r2_score(y_pred=predicciones_nn, y_true=y_test)

## Calibración

In [ ]:
calibracion(predicciones_nn, y_test)

# Conclusiones